In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_csv("./train.csv")

In [3]:
raw_data.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

%pip install contractions

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/chenzixuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chenzixuan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenzixuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached contractions-0.1.68-py2.py3-none-any.whl (8.1 kB)
  Using cached textsearch-0.0.21-py2.py3-none-any.whl (7.5 kB)
  Using cached anyascii-0.3.1-py3-none-any.whl (287 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring insta

In [10]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import contractions
import string

class Tokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        articles = articles.lower()

        for p in string.punctuation:
          articles = articles.replace(p, ' ')
        
        articles = contractions.fix(articles)
        
        res = []
        _stopwords = set(stopwords.words('english'))
        for t in word_tokenize(articles):
          token = self.wnl.lemmatize(t) 
          if token in _stopwords:
            continue
          res.append(token)
        return res

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import os

dict_name = "own_dict.txt"
# Try to load dictionary
if os.path.exists(dict_name):
    with open(dict_name, 'r') as f:
        own_dict = [word for word in f.readlines()]
else:
    # Build a dictionary according to word frequency
    vectorizer = CountVectorizer(tokenizer=Tokenizer())

    train_text = raw_data['question_text']
    labels = raw_data['target']

    train_vec = vectorizer.fit_transform(train_text)

    word_count = train_vec.sum(axis=0)
    word_count = np.asarray(word_count).reshape((1,-1))
    feature_names = vectorizer.get_feature_names_out().reshape(-1)
    word_count = pd.DataFrame(word_count, columns=feature_names)

    word_count.T.sort_values(by=0, ascending=False)

    word_count_sorted = word_count.T.sort_values(by=0, ascending=False)
    word_count_sorted.columns = ["count"]

    # filter all words that have <=150 frequency
    filter_index = sum(word_count_sorted["count"] > 150)
    own_dict = list(word_count_sorted.index[:filter_index])

    with open(dict_name, 'w') as f:
        for word in own_dict:
            f.write(word+'\n')

In [14]:
# Balance Dataset
pos_data = raw_data[raw_data['target'] == 0]
neg_data = raw_data[raw_data['target'] == 1]

sample_size = min(len(pos_data), len(neg_data))

pos_data = pos_data.sample(sample_size)
neg_data = neg_data.sample(sample_size)

train_data = pd.concat([pos_data, neg_data]).sample(frac=1)

train_data.head()
train_data

,qid,question_text,target
787627,9a4e197b444e8a341614,Why do Chinese Quora users strongly believe th...,1
1238013,f29d4bc51fee4bbb3631,Where does it clearly and specifically say in ...,1
621867,79c4ae9f40f9bbca1efa,"What is the meaning of TQWL7/WL4 in 3A, and wh...",0
155178,1e5a90b1c7833e020209,Why north Americans are less cultural than peo...,1
807188,9e2a320c121db8ac9ad6,What's one thing you did to get your life back...,0
...,...,...,...
145386,1c724fcc1ad47671e8d7,What are some of the ways God tests us nowaday...,0
298359,3a6c1fddf7de2e28b82b,By the way Indians and Chinese hate them selfs...,1
1123644,dc31da9587985c921fe2,Why are some people so blinded as to what kind...,1
947768,b9b87f20a1c7cbb6e33a,What are the benefits of eating celery leaves?,0


In [18]:
# vocab = {word: idx for idx, word in enumerate(own_dict) }
import pickle

new_vectorizer = CountVectorizer(tokenizer = Tokenizer())
new_vectorizer.fit_transform(own_dict)
train_vec = new_vectorizer.transform(train_data['question_text'])
labels = np.array(train_data['target'])

# save vectorizer
pickle.dump(new_vectorizer, open('./vectorizer', 'wb'))

print(train_vec.shape, labels.shape)

(161620, 6349) (161620,)


In [ ]:
# Train the model
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import time

X_train, X_test, y_train, y_test = train_test_split(train_vec, labels, test_size=0.15, random_state=42)

t1 = time.time()
svc = svm.SVC(random_state=42)
svc.fit(X_train, y_train)
t2 = time.time()

pred = svc.predict(X_test)

print(classification_report(y_test, pred))
print("training time:", (t2-t1))

In [ ]:
# Save the model
import pickle

model_name = "./svc.model"
pickle.dump(svc, open(model_name, 'wb'))
